# Data Analysis Of Medical Data

In [6]:
# Data Handling and Analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

### NEW
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report



In [7]:
# Read csv file and look at contents
file_path = "T49.2_Sep2025_1_StGallen.csv"
df = pd.read_csv(file_path, sep=";")

print(df.shape)
print(df.head())

(2310, 649)
   OUTCOME_3Kat_KHK           Alter_0  geschlecht waist_0             WHR_0  \
0                 2  56,9691991786448           0     112  ,982456140350877   
1                 2  69,4976043805613           0     104              1,04   
2                 2  72,0492813141684           0     129  ,941605839416058   
3                 2  70,8145106091718           0      99  ,876106194690266   
4                 2  62,1601642710472           1     125                     

              BMI_0  currsmo0 RR_syst_0 RR_diast_0 Pulse_Pressure  ...  \
0  32,8731097961867         0       130         80             50  ...   
1  30,1102788964583         0       160         70             90  ...   
2  44,7348800491207         0       140         90             50  ...   
3  26,2595847484332         0       130         90             40  ...   
4  42,4366343891054         1       180        100             80  ...   

  SMC231l SMC232l SMC233l SMC240l SMC241l SMC242l SMC243l SMC244l SM

In [10]:
# X / y
X = df.drop("OUTCOME_3Kat_KHK", axis=1)
y = df["OUTCOME_3Kat_KHK"]

# One-hot encode non-numeric columns (if any)
X = pd.get_dummies(X, drop_first=True)

# Train/test split (stratified for multiclass)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features (SVMs are sensitive to feature scale)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# Multiclass SVM (one-vs-one internally)
svm = SVC(kernel="rbf", C=1.0, gamma="scale", probability=False, verbose=True)  # verbose shows optimization progress
svm.fit(X_train_s, y_train)

# Evaluate
y_pred = svm.predict(X_test_s)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

[LibSVM].*
optimization finished, #iter = 1286
obj = -437.831794, rho = 0.080045
nSV = 794, nBSV = 291
.*.*
optimization finished, #iter = 2225
obj = -469.300479, rho = 0.552969
nSV = 1325, nBSV = 290
.*.*
optimization finished, #iter = 2330
obj = -851.485102, rho = 0.459630
nSV = 1558, nBSV = 535
Total nSV = 1848
Accuracy: 0.5692640692640693

Confusion Matrix:
 [[  0   0  73]
 [  0   0 126]
 [  0   0 263]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        73
           1       0.00      0.00      0.00       126
           2       0.57      1.00      0.73       263

    accuracy                           0.57       462
   macro avg       0.19      0.33      0.24       462
weighted avg       0.32      0.57      0.41       462



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:

# ---------- X / y ----------
y = df["OUTCOME_3Kat_KHK"].astype("category")
X = df.drop(columns=["OUTCOME_3Kat_KHK"])

# One-hot encode non-numeric columns (if any)
X = pd.get_dummies(X, drop_first=True)

# Optional: drop constant columns (can hurt SVM)
const_cols = [c for c in X.columns if X[c].nunique(dropna=False) <= 1]
if const_cols:
    X = X.drop(columns=const_cols)

# ---------- Split (stratified) ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train class distribution:\n", y_train.value_counts(normalize=True))
print("\nTest class distribution:\n", y_test.value_counts(normalize=True), "\n")

# ---------- Pipeline (impute ➜ scale ➜ SVM) ----------
pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),  # with_mean=False is safer for sparse matrices from get_dummies
    ("clf", SVC())  # placeholder; params below will set kernel & weights
])

# ---------- Param grid ----------
param_grid = [
    # RBF SVM (good default) with class balancing
    {
        "clf": [SVC(class_weight="balanced")],
        "clf__kernel": ["rbf"],
        "clf__C": [0.1, 1, 10, 100],
        "clf__gamma": ["scale", "auto", 0.1, 0.01, 0.001],
    },
    # Linear SVM as an alternative (often strong with many sparse features)
    {
        "clf": [LinearSVC(class_weight="balanced")],
        "clf__C": [0.1, 1, 10],
    }
]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="balanced_accuracy",   # better for imbalance
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("Best params:\n", grid.best_params_)
print("Best CV balanced-accuracy:", grid.best_score_)

# ---------- Evaluate on held-out test ----------
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)

print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Test Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print("\nConfusion Matrix (labels in order):")
labels = list(y.cat.categories)
print(confusion_matrix(y_test, y_pred, labels=labels))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, labels=labels))


Train class distribution:
 OUTCOME_3Kat_KHK
2    0.569264
1    0.273810
0    0.156926
Name: proportion, dtype: float64

Test class distribution:
 OUTCOME_3Kat_KHK
2    0.569264
1    0.272727
0    0.158009
Name: proportion, dtype: float64 

Fitting 5 folds for each of 23 candidates, totalling 115 fits


KeyboardInterrupt: 